In [1]:
print('importing pandas')
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
print('importing svm')
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

importing pandas
importing svm


In [2]:
print('defining vectorizer')
# Define the TF-IDF vectorizer with desired parameters
tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda x: [word_pos[0] for word_pos in eval(x)],lowercase=False,stop_words=None)

def tfidf_vectorize(data, column):
    # Fit and transform the specified column
    tfidf_vectors = tfidf_vectorizer.fit_transform(data[column])

    # Return the resulting sparse matrix
    return tfidf_vectors

defining vectorizer


In [3]:
def train_model(X, y):
    # Initialize a KFold object with 5 folds
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

    # Create lists to store the training history and trained models for each fold
    models = []
    
    # Create lists to store validation accuracy
    acc_avg_train = []
    acc_avg_test = []
    
    # Iterate over the folds and train the model on each one
    for fold, (train_indices, val_indices) in enumerate(kfold.split(X, y)):
        # Split the data into training and validation sets for this fold
        X_train, y_train = X[train_indices], y[train_indices]
        X_val, y_val = X[val_indices], y[val_indices]

        # Create a svm model
        model = svm.LinearSVC()

        # Train the model on the input data
        model.fit(X_train, y_train)

        # Add the trained model and training history to the lists
        models.append(model)
        
        # Validate
        accuracy_test = model.score(X_val, y_val)
        acc_avg_test.append(accuracy_test)
        accuracy_train = model.score(X_train, y_train)
        acc_avg_train.append(accuracy_train)
        

    #print average accuracy
    print('CV test accuracy:', sum(acc_avg_test)/len(acc_avg_test))
    print('CV train accuracy:', sum(acc_avg_train)/len(acc_avg_train))
    
    # Return the list of trained models and training histories
    return models

In [4]:
print('importing data')
df = pd.read_csv('/Users/christophershaffer/Documents/Hopkins/Theory of Machine Learning - EN.625.742/Project/News Category Dataset Cleaned 20230427.csv')

importing data


In [5]:
categories = ['WELLNESS & HEALTH','ENTERTAINMENT','TRAVEL', 'STYLE & BEAUTY', 'PARENTING', 'QUEER VOICES', 'FOOD & DRINK']
data = df[df['category'].isin(categories)]

sampled_data = data.groupby('category', group_keys=False).apply(lambda x: x.sample(n=3000))
print("data imported")
# Encode the target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(sampled_data['category'])
print("vectorize sparce")
# Vectorize the text data using TF-IDF
X_sparse = tfidf_vectorize(sampled_data, 'wordnet_pos_desc')

data imported
vectorize sparce


/Users/christophershaffer/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [6]:
X_sparse

<21000x31139 sparse matrix of type '<class 'numpy.float64'>'
	with 248916 stored elements in Compressed Sparse Row format>

In [7]:
print("data to array")
# Convert the sparse matrices to dense arrays
X = X_sparse.toarray()

data to array


In [8]:
print("train model")
# Train the support vector machine on the training data
models = train_model(X, y)

train model
CV test accuracy: 0.6288571428571428
CV train accuracy: 0.9431309523809525
